This is a starter notebook for the [Kitchenware classification](https://www.kaggle.com/competitions/kitchenware-classification) competition on Kaggle

To get started:

- Join the competition and accept rules
- Download your Kaggle credentials file
- If you're running in Saturn Cloud, configure your instance to have access to access the kaggle credentials

When this is done, we can download the data. We need to execute the following cell only once

In [1]:
# !kaggle competitions download -c kitchenware-classification
# !mkdir data
# !unzip kitchenware-classification.zip -d data > /dev/null
# !rm kitchenware-classification.zip

100%|█████████████████████████████████████▉| 1.63G/1.63G [00:23<00:00, 97.9MB/s]
100%|██████████████████████████████████████| 1.63G/1.63G [00:23<00:00, 75.6MB/s]


Now let's train a baseline model

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras

2022-12-16 07:37:23.857089: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


First, we will load the training dataframe and split it into train and validation

In [2]:
df_train_full = pd.read_csv('data/train.csv', dtype={'Id': str})
df_train_full['filename'] = 'data/images/' + df_train_full['Id'] + '.jpg'
df_train_full.head()

,Id,label,filename
0,0560,glass,data/images/0560.jpg
1,4675,cup,data/images/4675.jpg
2,0875,glass,data/images/0875.jpg
3,4436,spoon,data/images/4436.jpg
4,8265,plate,data/images/8265.jpg


In [3]:
val_cutoff = int(len(df_train_full) * 0.8)
df_train = df_train_full[:val_cutoff]
df_val = df_train_full[val_cutoff:]

Now let's create image generators

In [4]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   shear_range=10,
                                   zoom_range=0.1,
                                   horizontal_flip=True,
                                   vertical_flip=True
)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(299, 299),
    batch_size=32,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(299, 299),
    batch_size=32,
)

Found 4447 validated image filenames belonging to 6 classes.
Found 1112 validated image filenames belonging to 6 classes.


In [6]:
base_model = Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(299, 299, 3)
)
base_model.trainable = False

inputs = keras.Input(shape=(299, 299, 3))

base = base_model(inputs, training=False)
vectors = keras.layers.GlobalAveragePooling2D()(base)
outputs = keras.layers.Dense(6)(vectors)

model = keras.Model(inputs, outputs)

2022-12-16 07:38:35.534453: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 07:38:35.540906: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 07:38:35.541626: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 07:38:35.542832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [7]:
learning_rate = 0.001
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [8]:
history = model.fit(
    train_generator,
    epochs=2,
    validation_data=val_generator
)

Epoch 1/2


2022-12-16 07:39:06.887791: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-12-16 07:39:07.551988: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-16 07:39:07.552872: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-16 07:39:07.552915: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-12-16 07:39:07.553903: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-12-16 07:39:07.553998: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


139/139 [==============================] - 148s 1s/step - loss: 0.4994 - accuracy: 0.8774 - val_loss: 0.1891 - val_accuracy: 0.9532
Epoch 2/2
139/139 [==============================] - 135s 971ms/step - loss: 0.2189 - accuracy: 0.9395 - val_loss: 0.1513 - val_accuracy: 0.9577


Now let's use this model to predict the labels for test data

In [9]:
df_test = pd.read_csv('data/test.csv', dtype={'Id': str})
df_test['filename'] = 'data/images/' + df_test['Id'] + '.jpg'
df_test.head()

,Id,filename
0,0678,data/images/0678.jpg
1,3962,data/images/3962.jpg
2,9271,data/images/9271.jpg
3,5133,data/images/5133.jpg
4,8842,data/images/8842.jpg


In [11]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_dataframe(
    df_test,
    x_col='filename',
    class_mode='input',
    target_size=(299, 299),
    batch_size=32,
    shuffle=False
)

Found 3808 validated image filenames.


In [12]:
y_pred = model.predict(test_generator)

119/119 [==============================] - 60s 496ms/step


In [13]:
classes = np.array(list(train_generator.class_indices.keys()))
classes

array(['cup', 'fork', 'glass', 'knife', 'plate', 'spoon'], dtype='<U5')

In [14]:
predictions = classes[y_pred.argmax(axis=1)]

Finally, we need to prepare the submission

In [15]:
df_submission = pd.DataFrame()
df_submission['filename'] = test_generator.filenames
df_submission['label'] = predictions

df_submission['Id'] = df_submission.filename.str[len('data/images/'):-4]
del df_submission['filename']
df_submission

,label,Id
0,spoon,0678
1,knife,3962
2,fork,9271
3,plate,5133
4,fork,8842
...,...,...
3803,plate,7626
3804,cup,2052
3805,spoon,8827
3806,fork,2299


In [17]:
df_submission[['Id', 'label']].to_csv('submission.csv', index=False)

In [ ]:
!kaggle competitions submit kitchenware-classification -f submission.csv -m 'validation: 0.9577'

100%|██████████████████████████████████████| 38.8k/38.8k [00:00<00:00, 46.2kB/s]
